In [ ]:
import psycopg2                                                 
from psycopg2.extras import RealDictCursor                          
                                                            
conn = psycopg2.connect(host='localhost', user='postgres',          
password='Ember1808', database='baby_data')                         
cursor = conn.cursor(cursor_factory=RealDictCursor)                 
                                                            
print('=== BABY DATA DATABASE STRUCTURE ===\n')                     
                                                            
# Get all tables                                                    
cursor.execute(\"\"\"                                               
    SELECT table_name FROM information_schema.tables                
    WHERE table_schema = 'public' AND table_type = 'BASE TABLE'     
    ORDER BY table_name;                                            
\"\"\")                                                             
tables = cursor.fetchall()                                          
                                                            
for table in tables:                                                
    table_name = table['table_name']                                
    if table_name != 'alembic_version':  # Skip migration table     
        print(f'📋 TABLE: {table_name.upper()}')                    
        print('─' * 50)                                             
                                                            
        # Get column details                                        
        cursor.execute(\"\"\"                                       
            SELECT                                                  
                column_name,                                        
                data_type,                                          
                is_nullable,                                        
                column_default,                                     
                character_maximum_length                            
            FROM information_schema.columns                         
            WHERE table_name = %s AND table_schema = 'public'       
            ORDER BY ordinal_position;                              
        \"\"\", (table_name,))                                      
                                                            
        columns = cursor.fetchall()                                 
        for col in columns:                                         
            nullable = '(NULL)' if col['is_nullable'] == 'YES' else 
'(NOT NULL)'                                                        
            length = f'({col[\"character_maximum_length\"]})' if    
col['character_maximum_length'] else ''                             
            default = f' DEFAULT: {col[\"column_default\"]}' if     
col['column_default'] else ''                                       
            print(f'  • {col[\"column_name\"]:<20}                  
{col[\"data_type\"]}{length} {nullable}{default}')                  
                                                            
        print()                                                     
                                                            
conn.close()   